In [1]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration, LlavaForConditionalGeneration 
from huggingface_hub import login
import os

In [2]:
## memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

if 'model' in locals() or 'model' in globals():
    del model
torch.cuda.empty_cache()
print("allocated:")
torch.cuda.memory_allocated()
print("reserved:")
torch.cuda.memory_reserved()

allocated:
reserved:


0

In [3]:
# Read the token from a local file
with open("/home/mark/keys/hf_key.txt", "r") as f:
    hf_token = f.read().strip()

# Log in
login(hf_token)

In [4]:
## Set model_id here, to later load model and processor
#model_id = 'mistral-community/pixtral-12b'
model_id = 'meta-llama/Llama-3.2-11B-Vision-Instruct'


In [6]:
if model_id == "meta-llama/Llama-3.2-11B-Vision-Instruct":
    model = MllamaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        #use_cache=False
    )
    model = model.cuda()
    #model.gradient_checkpointing_enable()
    processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
if model_id == "mistral-community/pixtral-12b":
    processor = AutoProcessor.from_pretrained(model_id)
    model = LlavaForConditionalGeneration.from_pretrained(model_id).to("cuda")

In [8]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [10]:
image = Image.open('test_images/test_jpgs/pre/malawi-cyclone_00000000_pre_disaster.jpg')

In [13]:
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "How many man-made large buildings are in this satellite image? Do not count cars, sheds, or similarly small buildings."}
    ]}
]

input_text = processor.apply_chat_template(
    messages, add_generation_prompt=True,
)
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to(model.device)
output = model.generate(**inputs, max_new_tokens=70)

print(processor.decode(output[0][inputs["input_ids"].shape[-1]:]))

There are 8 man-made large buildings in this satellite image. There are 2 large buildings on the left side of the road, 2 more large buildings on the right side of the road, and 4 large buildings in the lower right corner of the image.<|eot_id|>
